In [6]:
# this library helps us manage data
import pandas as pd
# library for comprehending web documents for scraping
from bs4 import BeautifulSoup
# library for making requests to a webdomain for a web document
from urllib.request import urlopen
# regex library
import re
# to output and input csv data
import csv
# math library for random and uniform functions
from random import randint
from random import uniform
# native library to enable a script to sleep or wait
from time import sleep

In [3]:
#reading a list of items into a list
with open ("full_item_list.csv",'r') as f:
    reader=csv.reader(f)
    full_item_list=list(reader)

In [13]:
#lets take a look at the full item list
viewingpurposes=pd.DataFrame(full_item_list)
viewingpurposes.head()
#viewingpurposes.tail()

,0,1,2
0,Item_Name,Hero_Name,Item_Type
1,Acolyte of Vengeance Weapon,Anti-Mage,Equipment
2,Ancient Crasher,Anti-Mage,Equipment
3,Arc of Manta,Anti-Mage,Equipment
4,Basher of Mage Skulls,Anti-Mage,Equipment


I was told each item has an attribute called quality.
quality of an item is a prefix that denotes a unique version of the item that would have a different value. Not all items have all qualitites.

In [17]:
# create a list of all item qualities
quality = ['','Auspicious ', 'Heroic ', 'Genuine ','Frozen ', 'Cursed ', 'Base ', 'Exalted ', 'Infused ', 'Elder ', 'Legacy ', 'Unusual ', 'Corrupted ', 'Autographed ', 'Ascendant ', 'Inscribed ']

I intending on creating a script that will iterate through each item for each of its qualities and fetch the items:
1. price history
2. item rarity attribute

In [ ]:
#iterate through each item once
for i in range(1,len(full_item_list),1): 
    # each item has a list of unique prefixes(quality)
    for j in range(len(quality)):  
        # a full item name consists of prefix(quality) and word(item name)
        item_name=quality[j]+full_item_list[i][0]
        # urls keeps %20 as a seperator instead of spaces
        item_name=item_name.replace(" ", "%20")
        # each item corresponds to a particular hero
        # we fetch the hero name from the same line as the item name
        hero_name=full_item_list[i][1]
        # robots.txt denotes the rules of a website
        # scrapers usually incorporate some sort of random waiting in between each request
        # otherwise you can potentially be blacklisted from a website
        sleep(uniform(3.25,15.1)) #SLEEP TIMERS HERE
        # the url we are scraping is as follows
        # we format the url such that is dynamic to each request we are making
        url='https://steamcommunity.com/market/listings/570/{}'.format(item_name)
        #BeautifulSoup reads the web document and stores it as a BS Object
        soup=BeautifulSoup(urlopen(url), "lxml")
        # from the bs object we want to isolate a particular 
        # chunk of text with the data we are interested in
        scriptchunk=soup.find_all('script')[21].string
        #split the lines
        lines=scriptchunk.split('\n')
        #debugging mechanism for invalid requests
        print(item_name)
        if len(lines)<50:
            continue
        else:
            #Fetch item_name price history
            data1=lines[66]
            #regex magic and instantiating prices variable
            prices=re.findall(r"\[[^[]*[^]]\]", data1)
            #converting the regex isolated string to an array
            for j in range(len(prices)):
                prices[j] = eval(prices[j])
            
            ##GET item_rarity
            patterns = ['Common', 'Uncommon', 'Rare', 'Mythical','Immortal', 'Legendary', 'Arcana', 'Ancient']
            #i nstantiating an empty list where we append the items rarity
            # remember we want to output an items rarity as well as its price
            rarity = [] #setting up the list
            for pattern in patterns:
                #print('Looking for "%s" in text ->' % (pattern), end=' ')
                if re.search(pattern, lines[2]):
                    #print('found a match!')
                    rarity.append(1)
                else:
                    #print('no match')
                    rarity.append(0)
            #parsing the rarity list        
            total_sum = sum(rarity)
            if (total_sum ==1): #which is the default case
                item_rarity = patterns[rarity.index(1)]
                #print(item_rarity) 
            else:
                print("multiple rarities detected")
                item_rarity='error'
                
            #outputting out data
            with open("{}_{}.csv".format(hero_name,item_name),"w", newline='')as f:
                    writer=csv.writer(f)
                    writer.writerow(["Datetime","Price","Index No.","Item Rarity"])
                    for k in range(len(prices)):
                        output = []
                        for m in prices[k]:
                            output.append(m)
                        output.append(item_rarity)
                        writer.writerow(output)
            